<a href="https://colab.research.google.com/github/Jinzhao-Yu/BioStat615/blob/main/BIOSTAT615_Lecture_7_Fall_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BIOSTAT615 Lecture 7 - R

## 1. Bisection method for root finding

In [1]:
#' bisect0() - recursive implementation of bisection method
#' @param f : objective function to find root
#' @param a, b : two values with f(a) * f(b) < 0
#' @param tol : absolute difference between a,b for convergence
#' @return list containing the following attributes:
#'    * root - the x value with f(x) close to zero
#'    * f_root - f(root) value
bisect0 = function(f, a, b, tol=1e-10) {
  f.a = f(a); f.b = f(b)  ## evaluate f(a),f(b)
  if ( abs(b-a) < tol ) { # terminating condition
    root = (b+a)/2; 
    return (list(root=root, f_root=f(root)))
  } else {
    xmid = (a+b)/2; ymid = f(xmid) # obtain midpoint
    if ( f.a * ymid < 0 ) {  # divide-and-conquer
      return (bisect0(f, a, xmid, tol))
    } else {
      return (bisect0(f, xmid, b, tol))
    }
  }
}

Let's run bisection method to find the root on the following function
$$f(x) = e^x - x - 2$$

between $x \in [1,2]$ because we know that $f(1) = e - 3 < 0$ and $f(2) = e^2 - 4 > 0$.

In [2]:
bisect0(f=function(x) {exp(x)-x-2}, a=1,b=2,tol=1e-10)

$root
[1] 1.146193

$f_root
[1] -4.363132e-11

Recursive implementation can be sometimes slow. We can avoid repetitive function calls with loop-based implementation.

In [3]:
#' bisect1() - loopy implementation of bisection method
#' @param f : objective function to find root
#' @param a, b : two values with f(a) * f(b) < 0
#' @param tol : absolute difference between a,b for convergence
#' @param max_iter : maximum iterations allowed
#' @return list containing the following attributes:
#'    * root - the x value with f(x) close to zero
#'    * f_root - f(root) value
bisect1 <- function(f, a, b, tol=1e-10, max_iter = 100){
  iter <- 0
  f.a <- f(a)  ## evaluate initial value of f(a)
  f.b <- f(b)  ## evaluate initial value of f(b)
  convergence = 0
  if(f.a*f.b>0){  ## if f(a) and f(b) are the same sign, root-finding cannot be achieved.
    convergence = 2
    warning ("Initial conditions are not satisfied!")
  }
  while (abs(b - a) > tol) {  ## repeat the loop as long as a, b are not too close
    iter <- iter + 1        ## keep track of the number of iteration
    if (iter > max_iter) {
      convergence = 1
      warning (" Iterations maximum exceeded!")
      break
    }
    xmid <- (a + b) / 2     ## xmid is a midpoint between a and b
    ymid <- f(xmid) 
    if (f.a * ymid > 0) {   ## if f(a) and f(xmid) are the same sign
      a <- xmid             ## xmid is new a
      f.a <- ymid
    } else {
      b <- xmid;            ## otherwise, xmid is new b
    }
  }
  root <- (a + b)/2         
  f_root = f(root)
  return (list(root=root, f_root = f_root,iter=iter,convergence=convergence))
}

In [4]:
bisect1(f=function(x) exp(x)-x-2, a=1,b=2,tol=1e-10)

$root
[1] 1.146193

$f_root
[1] -4.363132e-11

$iter
[1] 34

$convergence
[1] 0

Let's try a simpler function $f(x) = x$ in $x \in [-1,1]$ to find a root

In [5]:
bisect1(f=function(x) x, a=-1,b=1,tol=1e-10)

$root
[1] -2.910383e-11

$f_root
[1] -2.910383e-11

$iter
[1] 35

$convergence
[1] 0

Are you happy with the algorithm? Could it be better?

## 2. Newton Raphson method for root finding

In [6]:
#' Newton.Raphson.root() - Newton-Raphson method for root finding
#' @param f : objective function to find root
#' @param fp : first-derivative of the function f()
#' @param x0 : initial starting point
#' @param tol : absolute difference of stepwise difference in x for convergence
#' @param max_iter : maximum number of iterations
#' @return A list containing the following attributes:
#'    * root - x value with f(x) close to zero
#'    * f_root - f(root)
#'    * iter - number of iterations to reach the solution
#'    * convergence - 0 if the root was found successfully, 1 if not found
Newton.Raphson.root <- function(f,fp,x0,tol=1e-10,max_iter=1000){
  convergence = 1 ## convergence=1 means convergence has not reached
  for(iter in 1:max_iter){
    fp0 = fp(x0) ## evaluate derivative
    if(abs(fp0)<tol){ ## if derivative is too small, it will be unstable
      convergence = 1
      warning("The first diverative gets close to zero!")
      break
    }
    f0 = f(x0)
    x1 = x0 - f0/fp0  ## next point to evaluate
    if(abs(x1-x0)<tol){ ## check convergence
      convergence = 0   ## convergence=0 means that convergence has reached.
      break
    }
    x0 = x1  ## update x0 and repeat the steps
  }
  return(list(root=x1,f_root = f(x1),iter=iter,convergence=convergence))
}

In [7]:
# objective function
f = function(x) exp(x)-x-2
# derivative of f - need to provide too
fp = function(x) exp(x)-1
Newton.Raphson.root(f=f,fp=fp,x0=1)

$root
[1] 1.146193

$f_root
[1] 0

$iter
[1] 5

$convergence
[1] 0

In [8]:
# objective function
f = function(x) exp(x)-x-2
# derivative of f
fp = function(x) exp(x)-1
Newton.Raphson.root(f=f,fp=fp,x0=-1)  ## start at a different point

$root
[1] -1.841406

$f_root
[1] 0

$iter
[1] 5

$convergence
[1] 0

Let's try [an example describe in Wikipedia](https://en.wikipedia.org/wiki/Newton%27s_method#Starting_point_enters_a_cycle). 

In [9]:
# objective function
f = function(x) x**3-2*x+2
# derivative of f
fp = function(x) 3*x*x-2
Newton.Raphson.root(f=f,fp=fp,x0=0)  ## start at x=0 or x=1

$root
[1] 0

$f_root
[1] 2

$iter
[1] 1000

$convergence
[1] 1

## 3. Secant method for root finding

In [10]:
#' secant() - secant method for root finding
#' @param f : objective function to find root
#' @param x0, x1 : initial starting points
#' @param tol : absolute difference of stepwise difference in x for convergence
#' @param max_iter : maximum number of iterations
#' @return A list containing the following attributes:
#'    * root - x value with f(x) close to zero
#'    * f_root - f(root)
#'    * iter - number of iterations to reach the solution
#'    * convergence - 0 if the root was found successfully, 1 if not found
secant <- function(f,x0,x1,tol=1e-10,max_iter=1000){
  convergence = 1
  f0 = f(x0); f1 = f(x1)
  if(abs(f0-f1)<tol){ ## if x1 and x0 are too close, approx derivative is near-zero.
    warning("Expect a huge jump!")
    break
  }
  x12 <- -f1/(f1-f0)*(x1-x0) ## change in x
  x2 <- x1 + x12             ## interpolation
  for(iter in 1:max_iter){
    if(abs(x12)<tol){
      convergence = 0 ## convergence has reached
      break
    }
    f0 <- f1 
    x1 <- x2
    f1 <- f(x2)
    f01 <- f1 - f0 ## difference of f(x)
    if(abs(f01)<tol){
      warning("Expect a huge jump!")
      break
    }
    x12 <- -f1/f01*x12 ## change in x - see equation in slide
    x2 <- x1 + x12     ## update rule.
  }
  return(list(root=x2,f_root = f(x2),iter=iter,convergence=convergence))
}

In [11]:
secant(f=f,x0=1,x1=2)

$root
[1] -1.769292

$f_root
[1] 0

$iter
[1] 25

$convergence
[1] 0

## 4. Finding a bracketing interval for minimization

In [12]:
#' bracket() - bracketing for 1-d minimization
#' @param f : objective function to find minimum point
#' @param a0, b0 : initial starting points
#' @param scale : ratio between step sizes
#' @param max_iter : maximum number of iterations
#' @return A list containing the following attributes:
#'   * intervals - triplets (a,b,c) where f(a)>f(b) and f(c)>f(b)
#'   * f_intervals - triples of (f(a), f(b), f(c))
#'   * iter - number of iterations to reach the solution
#'   * convergence - 0 if bracket was found successfully, 1 if not found
bracket <- function(f, a0, b0, scale=1.618, max_iter=1000){
  fa <- f(a0); fb <- f(b0) ## evaluate functions
  if ( fa < fb ) { ## if increasing, switch a and b
    tmp = a0; a0 = b0; b0 = tmp; ## swap a0, b0
    tmp = fa; fa = fb; fb = tmp; ## swap fa, fb
  }
  c0 <- b0 + scale*(b0 - a0) ## find the next point to iterate
  fc <- f(c0)
  iter = 1
  convergence = 0
  while(fb > fc || fb > fa){ ## repeat until fb is smallest value
    a0 <- b0; fa <- fb
    b0 <- c0; fb <- fc
    c0 <- b0 + scale*(b0 - a0)
    fc <- f(c0)
    if( (iter > max_iter) || is.infinite(fc) || is.infinite(c0)) { ## reached max iteration
      convergence = 1 ## a bracket was not found
      break
    }
    iter = iter+1
  }
  return(list(intervals=c(a0,b0,c0),f_intervals=c(fa,fb,fc),
              iter=iter,convergence=convergence))
}

In [13]:
Fun = function(x) exp(x)-0.5*x^2-2*x
print(bracket(Fun, 0, 1)) # works

$intervals
[1] 0.000 1.000 2.618

$f_intervals
[1] 1.0000000 0.2182818 5.0453176

$iter
[1] 1

$convergence
[1] 0



In [14]:
print(bracket(Fun, 1, 2)) # works

$intervals
[1]  2.000  1.000 -0.618

$f_intervals
[1] 1.3890561 0.2182818 1.5840594

$iter
[1] 1

$convergence
[1] 0



In [15]:
print(bracket(Fun, 0, 3)) # doesn't work

$intervals
[1] -7.368616e+153 -1.192242e+154 -1.929048e+154

$f_intervals
[1] -2.714825e+307 -7.107206e+307           -Inf

$iter
[1] 733

$convergence
[1] 1



## 5. The Golden Search Algorithm

In [16]:
#' golden.step() - find the next point to iterate for golden search
#' @param a0, b0, c0 : three points with f(a0)>f(b0) and f(c0)>f(b0)
#' @param gold : constant for golden ratio
#' @return The next x-coordinate to iterate for golden search
golden.step <- function(a0,b0,c0,gold=0.38196){
  mid = (a0+c0)*0.5 
  if(b0 > mid){   ## compare b0 and mid to decide the next point
    return(gold*(a0-b0)) 
  } else{
    return(gold*(c0-b0))
  }
}

In [17]:
#' golden.search() - golden section search algorithm
#' @param f : objective function to mimimize
#' @param a0, b0, c0 : initial points with f(a0)>f(b0) and f(c0)>f(b0)
#' @param max_iter : maximum iteration
#' @param tol : relative error for x-value
#'    * minimum - x value with f(x) being the minimum of f(x)
#'    * objective - f(minimum)
#'    * convergence - 0 if the minimum passed the convergence criteria, 1 if not
#'    * iter - number of iterations to reach the solution
#'    * tol - tol parameter same to input
golden.search <- function(f, a0, b0, c0, max_iter = 1000, tol=1e-8,
                          ...){
  fb = f(b0,...) ## In R, you can pass extra argument with ... between functions
  iter = 0
  convergence = 1 ## not yet converged
  while(iter < max_iter){
    x <- b0 + golden.step(a0,b0,c0) ## find the next step
    fx <- f(x,...) ## evaluate f(x)
    if(fx < fb){   ## x is the new minimum
      if(x > b0) a0 = b0 else c0 = b0
      b0 = x; fb = fx
    } else{        ## b0 is still the mimimum
      if(x < b0) a0 = x else c0 = x
    }
    iter = iter + 1
    if(abs(c0-a0) < abs(b0)*tol){ ## check convergence
      convergence = 0
      break
    }
  }
  return(list(minimum=b0,objective=fb,convergence=convergence,
              iter=iter,tol=tol))
}

In [18]:
Fun = function(x) exp(x)-0.5*x^2-2*x
print(golden.search(f = Fun,a0=-5,b0=0,c0=5))

$minimum
[1] 1.146193

$objective
[1] 0.1969273

$convergence
[1] 0

$iter
[1] 43

$tol
[1] 1e-08



## 6. Brent's method

In [19]:
Fun = function(x) exp(x)-0.5*x^2-2*x
print(optimize(f=Fun, c(-5,5), tol=1e-8))

$minimum
[1] 1.146193

$objective
[1] 0.1969273



In [20]:
## if minimum is at the boundary, no minima may be found.
print(optimize(f=Fun, c(-5,0), tol=1e-8))

$minimum
[1] -5

$objective
[1] -2.493262

